In [ ]:
#import the lib

import asyncio
from tweepy import Stream
from tweepy.streaming import StreamListener
from tweepy import OAuthHandler
from azure.eventhub.aio import EventHubProducerClient
from azure.eventhub import EventData
import json
import nltk
from textblob import TextBlob,Word,Blobber
nltk.download('punkt')
nltk.download('averaged_perceptron_tagger') 
nltk.download('brown')

In [ ]:
#this function used to analysis the data

def getblobsentiment(tweet):

  '''
  Utility function to classify sentiment of passed tweet
  '''
  # create TextBlob object of passed tweet text
  blob = TextBlob(tweet)
  i=0.00
  for sentence in blob.sentences:
      i=i+sentence.sentiment.polarity

  if i >0: 
    return 1
  elif i<0:
    return -1
  else:
    return 0

In [ ]:
#this function is used the make the connection with azure eventhub and twitter
async def run(text):
  producer = EventHubProducerClient.from_connection_string(conn_str="Endpoint=sb://analyticsexample.servicebus.windows.net/;SharedAccessKeyName=sh1;SharedAccessKey=Uv6v3T//ldATTMklhpfmG/tpO/FVy6E8pQR2oX4Kfhw=", eventhub_name="tweetacc")
  async with producer:
    # Create a batch.
    event_data_batch = await producer.create_batch()
    event_data_batch.add(EventData(text))
    await producer.send_batch(event_data_batch)  

In [ ]:
class listener(StreamListener): 
  '''
	Generic Twitter Class for sentiment analysis.
	'''
  def on_data(self, data): 
    loop = asyncio.get_event_loop()
    t=json.loads(data)
    d1 = {"text":t["text"] ,"sentiment": getblobsentiment(t["text"]) ,"created_at" :t["created_at"]  }
    print (type(data))     
    loop.run_until_complete(run(json.dumps(d1)))
    print(d1)
    return(True)
   
    
  def on_error(self, status):
    print(status)
    
ckey="" 
csecret="" 
atoken="" 
asecret=""

auth = OAuthHandler(ckey, csecret) 
auth.set_access_token(atoken, asecret)

twitterStream = Stream(auth, listener()) 
twitterStream.filter(track=["corona"])